---
## Loading the `spag` package
---

In [ ]:
## Originally used for Script Testing, not Python Notebooks
import spag
from spag.convert import *
from spag.utils_data import *
from spag.utils_plotting import *
from spag import read_data as rd

print("spag version:", spag.__version__)
print("logeps(H) = "', ' rd.solar_logepsX_asplund09(Z=1))


In [1]:
import numpy as np
import pandas as pd

from spag import read_data as s_rd
from alexmods import read_data as a_rd

from pprint import pprint

# from astropy.utils.exceptions import AstropyDeprecationWarning
# import warnings
# warnings.filterwarnings("ignore", category=AstropyDeprecationWarning)

# from spag.utils_plotting import *
from spag.utils_data import *
# from spag.convert import *
# import spag.periodic_table as pt
from spag.periodic_table import pt_list, pt_dict
# from spag.read_data import *

Could not import continuum GUI (you are probably in python 2)


        Use @ instead. [gala.coordinates.sgr]
        Use @ instead. [gala.coordinates.orphan]
        Use @ instead. [gala.coordinates.magellanic_stream]


In [ ]:
asplund09 = s_rd.solar_logepsX_asplund09()
asplund09.columns
s_rd.load_jinabase(key=None,p=1,load_eps=True,load_ul=False,load_XH=True,load_XFe=True,load_aux=True,
                  name_as_index=False)




Loading Datafile:  /Users/ayelland/Research/metal-poor-stars/spag/spag/data/solar/asplund2009_solarabundance_table1.csv


AttributeError: 'DataFrame' object has no attribute 'as_matrix'

In [ ]:
## Default parameters
sci_key=None ## options: [nan, 'R2', 'R1', 'S', 'I']
priority=1 ## options: [1, 2], but can use 0 to keep all entries
load_eps=True ##
load_ul=True ##
load_XH=True
load_XFe=True
load_aux=True ##
name_as_index=True ##

data_dir = '/Users/ayelland/Research/metal-poor-stars/spag/spag/data/'

## Read data
data = pd.read_csv(data_dir+"abundance_tables/JINAbase-updated/JINAbase_2021_yelland.csv", header=0, na_values=["*"]) #index_col=0
uls  = pd.read_csv(data_dir+"abundance_tables/JINAbase-updated/JINAbase_2021_ulimits.csv", header=0, na_values=["*"]) #index_col=0
Nstars = len(data)

## Get the list of elements & create the corresponding column names for the element abundance columns
## NOTE: The ionization state columns are dropped and not used in this analysis. Hence, the -7 
##       in the column slicing. Every application of elems, ul_elems, and epscolnames now 
##       excludes the ionization state columns.
elems = data.columns[26:-7]
ul_elems = list(map(lambda x: "ul"+x, elems))
epscolnames = list(map(lambda x: "eps"+x.lower(), elems))
print("WARNING: Dropped the CaII, TiII, VII, CrII, MnII, FeII columns.")

## Rename the data element abundance columns with the prefix "eps" (e.g. "Fe" -> "epsfe")
data.rename(columns=dict(zip(elems, epscolnames)), inplace=True)

## Separate the auxiliary columns (JINA_ID, Priority, etc.) from the element abundance columns (epsX) in 'data' and 'uls'
auxdata_cols = data.columns[0:26].append(pd.Index([data.columns[-1]]))
auxdata = data[auxdata_cols]
data = data[epscolnames]

auxuls_cols = uls.columns[0:5]
auxuls = uls[auxuls_cols]
uls = uls[ul_elems] # consist of 1s (upper limits) and NaNs (non-upper limits)

## Use the ulimits (uls) DataFrame to mask the data DataFrame
uls_mask = pd.notnull(uls).to_numpy()  # Convert uls DataFrame to boolean array (True if not NaN)
uls_table = data.where(uls_mask)  # Extract only the upper limit values (keep NaN for others)
for col in uls_table.columns:
    uls_table.rename(columns={col: "ul"+col[3:]}, inplace=True) #same as ulcolnames (e.g. "ulFe" -> "ulfe")

data_matrix = data.to_numpy()  # Convert data DataFrame to NumPy array
data_matrix[uls_mask] = np.nan # Set values in `data_matrix` to NaN wherever `uls_mask` is True
data = pd.DataFrame(data_matrix, columns=data.columns, index=data.index) # Convert the modified NumPy array back to a DataFrame

## Concatenate the upper limit values to the 'data' DataFrame
if load_ul:
    data = pd.concat([data, uls_table], axis=1) # Concatenate the upper limit values to the data DataFrame

## Convert the element abundance and add the [X/H] and [X/Fe] columns
if load_XH:
    XH_from_eps(data)
if load_XFe:
    XFe_from_eps(data)

## Use a specific `Sci_key` to filter the data
if sci_key != None:
    assert sci_key in auxdata["Sci_key"].unique(), auxdata["Sci_key"].unique() #if sci_key is not in the list of unique Sci_keys, print the list of unique Sci_keys
    auxdata = auxdata[auxdata["Sci_key"]==sci_key]
else:
    assert len(data) == Nstars, (len(data), Nstars)

## Combine the auxiliary columns with the element abundance columns
if load_aux:
    data = pd.concat([data, auxdata],axis=1)
    # Remove duplicate entries by using the "Priority" column (1=high, 2=low)
    # (dupicates entries originate from two papers referencing the same source)
    if priority==1 or priority==2:
        pmask = data["Priority"] == priority
        data = data[pmask]
        uls = uls[pmask]
else:
    data = pd.concat([data, auxdata[["Name","Ref","Priority"]]], axis=1)
    # Remove duplicate entries by using the "Priority" column (1=high, 2=low)
    # (dupicates entries originate from two papers referencing the same source)
    if priority==1 or priority==2:
        pmask = data["Priority"] == priority
        data = data[pmask]
        uls = uls[pmask]
    data.drop("Priority", axis=1, inplace=True)

## Drop the log(eps) columns if not needed
if not load_eps:
    data.drop(eps_elems, axis=1, inplace=True)

## Set the "Name" column as the index
if name_as_index:
    data.index = data["Name"]


display(data)
display(uls)



# save the data file
data.to_csv(data_dir+"abundance_tables/JINAbase-updated/JINAbase_2021_yelland_processed.csv", index=False)
uls_table.to_csv(data_dir+"abundance_tables/JINAbase-updated/JINAbase_2021_yelland_uls_processed.csv", index=False)




Loading Datafile:  /Users/ayelland/Research/metal-poor-stars/spag/spag/data/solar/asplund2009_solarabundance_table1.csv


,epsli,epsbe,epsc,epsn,epso,epsf,epsna,epsmg,epsal,epssi,...,R_mag,I_mag,J_mag,H_mag,K_mag,Teff,logg,Fe/H,Vmic,Added_by
Name,,,,,,,,,,,,,,,,,,,,,
S10-14,NaN,NaN,NaN,NaN,NaN,NaN,2.79,4.77,NaN,NaN,...,NaN,16.56,NaN,NaN,15.08,4620.0,1.20,-2.70,2.20,Abdu
S11-37,NaN,NaN,NaN,NaN,NaN,NaN,3.32,4.67,NaN,NaN,...,NaN,16.90,NaN,NaN,15.34,4560.0,1.30,-2.90,2.20,Abdu
S12-28,NaN,NaN,NaN,NaN,NaN,NaN,3.22,4.9,NaN,NaN,...,NaN,16.44,NaN,NaN,14.91,4570.0,1.40,-2.80,2.70,Abdu
S14-98,NaN,NaN,NaN,NaN,NaN,NaN,3.45,4.78,NaN,NaN,...,NaN,17.06,NaN,NaN,15.61,4730.0,1.10,-2.70,2.80,Abdu
S15-19,NaN,NaN,NaN,NaN,NaN,NaN,3.25,4.86,NaN,NaN,...,NaN,16.53,NaN,NaN,14.98,4620.0,1.20,-3.10,2.60,Abdu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4980.0,1.82,-2.62,1.58,Mohammad
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5292.0,2.56,-2.79,1.53,Mohammad
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5153.0,2.20,-2.70,1.55,Mohammad


,ulLi,ulBe,ulC,ulN,ulO,ulF,ulNa,ulMg,ulAl,ulSi,...,ulW,ulOs,ulIr,ulPt,ulAu,ulHg,ulPb,ulBi,ulTh,ulU
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
